In [219]:
import html
import json
import os
import random
import requests

from bs4 import BeautifulSoup
from datetime import datetime, timedelta

from dotenv import load_dotenv

from openai import OpenAI
from openai.types.chat import ChatCompletion

from typing import List, Tuple
from slugify import slugify


In [189]:
load_dotenv()
OPENAI_SECRET_KEY = os.getenv('OPENAI_SECRET_KEY')

In [200]:
class OpenAi():
	def __init__(self, api_key):
		self.client = OpenAI(api_key=api_key)
	
	def generate_title(self, url: str) -> str:
		response = self.client.chat.completions.create(
			model='gpt-3.5-turbo',
			max_tokens=4096,
			messages=[
				{'role': 'system', 'content': 'You are an SEO expert. Given a url your task is it to find a title for a blog page that will rank well in search engines. json format: {"title": "title"}'},
				{'role': 'user', 'content': 'https://coinflip.tech/blog/unraveling-the-fed-market-news-bitcoin-moves'}
			],
			response_format={'type': 'json_object'},
		)
		json_response = response.choices[0].message.content.strip()
		return json.loads(json_response)['title']

	def generate_sub_titles(self, title: str) -> List[str]:
		response = self.client.chat.completions.create(
			model='gpt-3.5-turbo',
			max_tokens=4096,
			messages=[
				{'role': 'system', 'content': 'You are an SEO expert and write blog posts which are SEO optimized. A title for a blog post, generate 6 section subtitles. json format: {"titles": ["title1", "title2", "title3", "title4", "title5", "title6" ]}'},
				{'role': 'user', 'content': title}
			],
			response_format={'type': 'json_object'},
		)

		json_response = response.choices[0].message.content.strip()
		return json.loads(json_response)['titles']
	
	def relevant_sitemap(self, sitemap_xml: str) -> bool:
		response = self.client.chat.completions.create(
			model='gpt-3.5-turbo',
			max_tokens=128,
			messages=[
				{'role': 'system', 'content': 'You are an SEO expert. Given a links of a sitemap you have to say True or False to weather the pages on sitemap as a whole is relevant for page seo. respond with "True" for relevant and "False" for irrelevant. json format: {"answer": "True"}'},
				{'role': 'user', 'content': sitemap_xml}
			],
			response_format={'type': 'json_object'},
		)
		json_response = response.choices[0].message.content.strip()
		return json.loads(json_response)['answer'] == 'True'
	
	def genearte_first_section(self, title: str, sub_title: str) -> str:
		response = self.client.chat.completions.create(
			model='gpt-4-0125-preview',
			max_tokens=4096,
			messages=[
				{'role': 'system', 'content': 'You are an SEO expert. Your task is it to write a section for a blog. You will be given the title and sub titles of each section and your task is it to write the introduction, that is the first section which introduces the article. Return it in html format. The article must be SEO friendly and do well for keywords. Do not create a div wrapper which wraps the whole article. Json format: {"article": "article in html format"}'},
				{'role': 'user', 'content': f'title: {title}, sub_titles: {sub_title}'}
			],
			response_format={'type': 'json_object'},
		)
		json_response = response.choices[0].message.content.strip()
		return json.loads(json_response)['article']
	
	def generate_section(self, title: str, sub_title: str) -> str:
		response = self.client.chat.completions.create(
			model='gpt-4-0125-preview',
			max_tokens=4096,
			messages=[
				{'role': 'system', 'content': 'You are an SEO expert. Your task is it to write a section for a blog, you are only writing a section of a blog, not the whole blog. Therefore do not create a summary or conclusion in the end if you do I will die. You will be given the title of the whole blog and the title of the section. The section you write should focus on the sub title without summarizing or concluding the overall argument. Never write a conclusion. Return the section in html format. It must be 600 to 700 words. The article must be SEO friendly and do well for keywords. Do not create a div wrapper which wraps the whole article. Json format: {"article": "article in html format"}'},
				{'role': 'user', 'content': f'title: {title}, sub_title: {sub_title}'}
			],
			response_format={'type': 'json_object'},
		)
		json_response = response.choices[0].message.content.strip()
		return json.loads(json_response)['article']
	
	def generate_conclusion(self, article: str) -> str:
		response = self.client.chat.completions.create(
			model='gpt-4-0125-preview',
			max_tokens=4096,
			messages=[
				{'role': 'system', 'content': 'You are an SEO expert. Your task is it to write a conclusion given the article. Return the section in html format. It must be 600 to 700 words. The article must be SEO friendly and do well for keywords. Do not create a div wrapper which wraps the whole article. Json format: {"article": "article in html format"}'},
				{'role': 'user', 'content': f'Article: {article}'}
			],
			response_format={'type': 'json_object'},
		)
		json_response = response.choices[0].message.content.strip()
		return json.loads(json_response)['article']
	

In [201]:
openai = OpenAi(os.getenv('OPENAI_SECRET_KEY'))

In [192]:
sitemaps_to_check = ['https://coinflip.tech/sitemap.xml']
articles = []

In [193]:
def sitemap_index(url: str) -> Tuple[bool, List[str]]:
	"""
	Checks if the sitemap is a sitemap index or a sitemap
	returns True if it is a sitemap index
	"""
	response = requests.get(url)
	soup = BeautifulSoup(response.content, 'xml')
	if soup.find('sitemapindex'):
		return (True, [loc.text for loc in soup.find_all('loc')])
	elif soup.find('urlset'):
		locs = [loc.text for loc in soup.find_all('loc')]
		return (False, locs if openai.relevant_sitemap(f"[{', '.join(locs)}]") else [])
	else:
		raise Exception('Invalid sitemap')

In [194]:
while sitemaps_to_check:
	url = sitemaps_to_check.pop()
	is_sitemap_index, urls = sitemap_index(url)
	if is_sitemap_index:
		sitemaps_to_check.extend(urls)
	else:
		articles.extend(urls)

In [195]:
articles

['https://coinflip.tech/en-CA',
 'https://coinflip.tech/pt-BR',
 'https://coinflip.tech/es-PA',
 'https://coinflip.tech/en-ZA',
 'https://coinflip.tech/en-AU',
 'https://coinflip.tech/en-NZ',
 'https://coinflip.tech/blog/unraveling-the-fed-market-news-bitcoin-moves',
 'https://coinflip.tech/blog/correlations-between-bitcoin-and-the-nasdaq',
 'https://coinflip.tech/blog/coinflip-preferred-q1-recap',
 'https://coinflip.tech/blog/bitcoin-halving',
 'https://coinflip.tech/blog/the-ebb-and-flow-of-bitcoin-in-turbulent-economic-times',
 'https://coinflip.tech/blog/cryptocurrency-explained-stellar',
 'https://coinflip.tech/blog/cryptocurrency-explained-binance',
 'https://coinflip.tech/blog/cryptocurrency-explained-what-is-pax-gold',
 'https://coinflip.tech/blog/beware-the-hidden-fees',
 'https://coinflip.tech/blog/when-is-bitcoin-pizza-day-and-why-do-crypto-fans-care',
 'https://coinflip.tech/blog/diversifying-your-portfolio-with-crypto-and-diversifying-your-crypto',
 'https://coinflip.tech/

In [256]:
def generate_article(url: str):
	title = openai.generate_title(url)
	sub_titles = openai.generate_sub_titles(title)
	description = openai.genearte_first_section(title, ', '.join(sub_titles))
	article = description
	description = BeautifulSoup(description, 'html.parser')
	description = description.p.text[:300]
	for sub_title in sub_titles:
		article += openai.generate_section(title, sub_title)
	article += openai.generate_conclusion(article)
	return {'title': title, 'slug': slugify(title), 'description': description, 'article': article} 

In [303]:
article = generate_article('https://coinflip.tech/blog/bitcoin-halving')

In [319]:
def escape_html(s):
	return (
		s.replace("&", "&amp;")
		.replace("<", "&lt;")
		.replace(">", "&gt;")
		.replace("'", "&apos;")
		.replace('"', "&quot;")
	)

temp = article
print(temp)
soup = BeautifulSoup(temp['article'], 'html.parser')
first_h1 = soup.find('h1')
# If the first tag in the soup is a h1 tag, remove it
if soup.contents[0] == first_h1:
	first_h1.decompose()

for tag in soup():
	if 'class' in tag.attrs:
		del tag['class']
	if 'classname' in tag.attrs:
		del tag['classname']
	# Add 'className' attribute
	tag['className'] = 'blogContent'
	if tag.string is not None:
		tag.string = escape_html(tag.string.replace('’', "'"))
article['article'] = soup.prettify().replace("&amp;", "&").replace("&amp;", "&")
article['slug'] = slugify(article['title'])

{'title': 'Unraveling the Fed: Market News and Bitcoin Moves | CoinFlip Blog', 'slug': 'unraveling-the-fed-market-news-and-bitcoin-moves-coinflip-blog', 'description': "Welcome to our latest exploration on the CoinFlip Blog, where we delve into the intricate dance between the Federal Reserve's policies, market news, and the turbulent world of Bitcoin. In today's financial landscape, understanding the Federal Reserve's influence is more crucial than ever. Its decisi", 'article': '<h1 className="blogContent">\n Unraveling the Fed: Market News and Bitcoin Moves | CoinFlip Blog\n</h1>\n<p className="blogContent">\n Welcome to our latest exploration on the CoinFlip Blog, where we delve into the intricate dance between the Federal Reserve&amp;apos;s policies, market news, and the turbulent world of Bitcoin. In today&amp;apos;s financial landscape, understanding the Federal Reserve&amp;apos;s influence is more crucial than ever. Its decisions on interest rates and monetary policies send rippl

In [320]:
def random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2024, 4, 10)):
	random_date = start_date + (end_date - start_date) * random.random()
	# Format the date as a string in the format YYYY-MM-DD
	return random_date.strftime('%Y-%m-%d')

In [321]:
def create_nextjs_page(article: dict):
	return """
import {{ Metadata }} from 'next';

export const metadata: Metadata = {{
	metadataBase: new URL('https://mooncoinflip.com'),
	title: "{title}",
	description: "{description}",
	keywords: ['crypto', 'coinflip', 'casino', 'crypto casino', 'crypto coinflip', 'crypto gambling', 'gambling', 'gamble', 'crypto gamble', 'crypto gambling', 'crypto games', 'crypto game', 'crypto', 'bnb', 'binance smart chain', 'bsc', 'ethereum', 'double crypto'],
	icons: {{
		icon: 'https://mooncoinflip.com/imgs/icon.png',
		shortcut: 'https://mooncoinflip.com/imgs/logo_big.png',
		apple: 'https://mooncoinflip.com/imgs/icon.png',
	}},
	openGraph: {{
		title: "{title}",
		description: "{description}",
		url: 'https://mooncoinflip.com/blog/{slug}',
		siteName: "{title}",
		images: [
			{{
				url: 'https://mooncoinflip.com/blog/imgs/{slug}/header.webp',
				width: 1200,
				height: 800,
			}},
		],
		locale: 'en_US',
		type: 'website',
	}},
	twitter: {{
		card: 'summary_large_image',
		title: "{title}",
		description: "{description}",
		// siteId: '1467726470533754880',
		creator: '@mooncoinflipcom',
		// creatorId: '1467726470533754880',
		images: ['https://mooncoinflip.com/blog/imgs/{slug}/header.webp'],
	}},
	robots: {{
		index: true,
		follow: true,
		nocache: true,
		googleBot: {{
			index: true,
			follow: true,
		}},
	}},
}}

export default function Page() {{
	return <>
		<div className='flex justify-between flex-col md:flex-row'>
			<h1 className='mb-4'>
				{title_html}
			</h1>
			<p className='text-right mb-4'>
				Published: {date}
			</p>
		</div>
		<div>
			{article}
		</div>
	</>
}}
	""".format(
		title=article['title'],
		slug=article['slug'],
		description=article['description'], 
		title_html=html.escape(article['title']),
		date=random_date(),
		article=article['article']
	)

In [322]:
with open('output.tsx', 'w') as f:
	# Write the result to the file
	f.write(create_nextjs_page(article).strip().replace('’', "'"))